In [49]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Full path to your Excel file
excel_file = r"C:\Users\ktaj1\OneDrive - Louisiana State University\UHPC.xlsx"
df = pd.read_excel(excel_file, sheet_name='Sheet2')

# Extract columns from the DataFrame
A = df['Cement'].values
B = df['Silica Fume'].values
C = df['Sand'].values
D = df['Slag'].values
alpha = df['Optimal'].values

# Define densities for the components (g/cm^3)
Density_OPC = 3.15
Density_SF = 2.2
Density_SS = 2.6
Density_Slag = 2.85

# Fixed value for Weight_OPC
Weight_OPC = 1.0

# Define the RMSE function to minimize
def rmse_function(weights):
    Weight_SF, Weight_SS, Weight_Slag = weights
    Total_Volume = (Weight_OPC / Density_OPC 
                    + Weight_SF / Density_SF 
                    + Weight_SS / Density_SS 
                    + Weight_Slag / Density_Slag)
    
    # Calculate Binder based on the provided formula
    Binder = (A * Weight_OPC / (Total_Volume * Density_OPC)
              + B * Weight_SF / (Total_Volume * Density_SF)
              + C * Weight_SS / (Total_Volume * Density_SS)
              + D * Weight_Slag / (Total_Volume * Density_Slag))
    
    # Create a cumulative column based on Binder
    cumulative = np.cumsum(Binder)

    # Save cumulative to be used for MAE calculation
    rmse_function.cumulative = cumulative
    rmse = np.sqrt(np.mean((alpha[4:100] - cumulative[4:100])**2))
    return rmse

# Initial guess for the weights [Weight_SF, Weight_SS, Weight_Slag]
initial_guess = [0.1, 1.0, 1.0 / 10]  # An initial guess within the acceptable range for Weight_Slag

# Function to dynamically set bounds for weights
def get_bounds():
    return [(0, 0.2), (0, None), (0.091, 0.15)]

# List of optimization methods that support bounds
methods = ['L-BFGS-B', 'TNC', 'SLSQP']

for method in methods:
    bounds = get_bounds()
    result = minimize(rmse_function, initial_guess, bounds=bounds, method=method)
    
    # Extract the optimal weights
    optimal_weights = result.x
    
    # Calculate the cumulative Binder values for the optimal weights
    cumulative = rmse_function.cumulative
    
    # Calculate MAE
    mae = np.mean(np.abs(alpha[4:100] - cumulative[4:100]))
    
    # Print the method used, optimal weights, minimum RMSE, and MAE
    print(f"Method: {method}")
    print("Optimal weights (with Weight_OPC fixed at 1.0):")
    print("Weight_OPC = {:.3f}".format(Weight_OPC))
    print("Weight_SF = {:.3f}".format(optimal_weights[0]))
    print("Weight_SS = {:.3f}".format(optimal_weights[1]))
    print("Weight_Slag = {:.3f}".format(optimal_weights[2]))
    print("Minimum RMSE = {:.3f}".format(result.fun))
    print("MAE = {:.3f}".format(mae))
    



Method: L-BFGS-B
Optimal weights (with Weight_OPC fixed at 1.0):
Weight_OPC = 1.000
Weight_SF = 0.200
Weight_SS = 0.807
Weight_Slag = 0.091
Minimum RMSE = 4.283
MAE = 3.721
Method: TNC
Optimal weights (with Weight_OPC fixed at 1.0):
Weight_OPC = 1.000
Weight_SF = 0.200
Weight_SS = 0.807
Weight_Slag = 0.091
Minimum RMSE = 4.283
MAE = 3.721
Method: SLSQP
Optimal weights (with Weight_OPC fixed at 1.0):
Weight_OPC = 1.000
Weight_SF = 0.200
Weight_SS = 0.807
Weight_Slag = 0.091
Minimum RMSE = 4.283
MAE = 3.721
